# <span style='font-family:"Times New Roman"'> <span styel=''> **MASTER FILE CREATION**

*Emile Cohen* 
    
*March 2020*

**Goal:** In this Notebook, we create a master file that summarizes all useful information.

The Notebook is divided in 4 parts, representing the four parts of our Master file:
   
* **1. Patient/Sample Information**
* **2. TP53 Mutations**
* **3. TP53 Copy Numbers**
* **4. TP53 Computed Metrics**
* **5. Subgroup columns creation**
* **6. Merge tables**

**NB1:** In each part, you must run the cells from the begining in order to initialize the variables

**NB2:** In order to launch the last script (Merge Tables), you have to define the functions in each part.

**NB3:** All functions used for the plots are located in utils/custom_tools.py

---

In [4]:
%run -i '../../../utils/setup_environment.ipy'

import warnings
warnings.filterwarnings('ignore')
from scipy.stats import fisher_exact, ranksums, chi2, norm
from statsmodels.sandbox.stats.multicomp import multipletests
import matplotlib.gridspec as gridspec
import pickle

data_path = '../../../data/'
data_wgd = data_path + 'impact-facets-tp53/processed/wgd/'
data_no_wgd = data_path + 'impact-facets-tp53/processed/no_wgd/'

Setup environment... done!


<span style="color:green">✅ Working on **mskimpact_env** conda environment.</span>

In [82]:
# first we load all files for WGD
maf_cohort_wgd = pd.read_csv(data_wgd + 'maf_cohort_wgd.txt', sep='\t').drop('Unnamed: 0', axis=1)
cohort_wgd = pd.read_csv(data_wgd + 'cohort_wgd.txt', sep='\t').drop('Unnamed: 0', axis=1)
arm_level_wgd = pd.read_csv(data_wgd + 'arm_level_wgd.txt', sep='\t').drop('Unnamed: 0', axis=1)
gene_level_wgd = pd.read_csv(data_wgd + 'gene_level_wgd.txt', sep='\t').drop('Unnamed: 0', axis=1)

In [83]:
# We load all files for non WGD

maf_cohort_nowgd = pd.read_csv(data_no_wgd + 'maf_cohort_nowgd.txt', sep='\t').drop('Unnamed: 0', axis=1)
cohort_nowgd = pd.read_csv(data_no_wgd + 'cohort_nowgd.txt', sep='\t').drop('Unnamed: 0', axis=1)
arm_level_nowgd = pd.read_csv(data_no_wgd + 'arm_level_nowgd.txt', sep='\t').drop('Unnamed: 0', axis=1)
gene_level_nowgd = pd.read_csv(data_no_wgd + 'gene_level_nowgd.txt', sep='\t').drop('Unnamed: 0', axis=1)

In [84]:
# Creating keys for mutations in maf files

# First we need to create a sample_mut_key to identify duplicated mutations
maf_cohort_wgd['mut_key'] = maf_cohort_wgd.apply(lambda h: str(h.Chromosome)+'_'+str(h.Start_Position)+'_'+str(h.Reference_Allele)+'_'+str(h.Tumor_Seq_Allele2), axis=1) 
maf_cohort_nowgd['mut_key'] = maf_cohort_nowgd.apply(lambda h: str(h.Chromosome)+'_'+str(h.Start_Position)+'_'+str(h.Reference_Allele)+'_'+str(h.Tumor_Seq_Allele2), axis=1) 

# Create a sample key to differentiate duplicates
maf_cohort_wgd['sample_mut_key'] = maf_cohort_wgd.apply(lambda h: h.Tumor_Sample_Barcode + h.mut_key, axis = 1)
maf_cohort_nowgd['sample_mut_key'] = maf_cohort_nowgd.apply(lambda h: h.Tumor_Sample_Barcode + h.mut_key, axis = 1)

In [22]:
# Load clinical data
clinical_data = pd.read_csv(data_path + 'cbioportal/raw/mskimpact_clinical_data-2.tsv', sep= '\t')

In [34]:
# Filtering the clinical data
samples_wgd = list(set(cohort_wgd.tumor_sample))
samples_nowgd = list(set(cohort_nowgd.tumor_sample))

clinical_wgd = clinical_data[clinical_data['Sample ID'].isin(samples_wgd)]
clinical_nowgd = clinical_data[clinical_data['Sample ID'].isin(samples_nowgd)]

# IMPORTANT: Parameter Definition
In this script we can create two master files: one for WGD samples and one for non WGD samples.
So this parameter allows to select the type of master you want.
On top of that you can specify where you want to store the file you created

In [146]:
cohort_type = 'wgd'
output_path = data_path + 'impact-facets-tp53/processed/'

total_output = output_path + cohort_type + '/'

---
# Patient/Sample Information

In this part, we focus on clinical information exported from CbioPortal.

The following columns are selected:
* Sample_Id
* Tumor_Id
* Patient_Id
* Patient Current Age
* Cancer_Type
* Cancer_Type_Detailed
* Sample_Type
* purity
* ploidy
* Overall Survival Status
* Overall Survival (Months)
* MSI Score
* MSI Type
* Tumor Mutational Burden

In [77]:
def create_sample_info(cohort:str):
    if cohort == 'wgd':
        cohort = cohort_wgd
        maf_cohort = maf_cohort_wgd
        clinical = clinical_wgd
    elif cohort == 'no_wgd':
        cohort = cohort_nowgd
        maf_cohort = maf_cohort_nowgd
        clinical = clinical_nowgd

    cohort_filt = cohort[['sample_id', 'tumor_sample', 'patient', 'ploidy']]
    clinical_filt = clinical[['Sample ID', 'Patient Current Age', 'Cancer Type', 'Cancer Type Detailed', 'Sample Type',
                  'Overall Survival (Months)', 'Overall Survival Status','MSI Score', 'MSI Type','Impact TMB Score']]
    purity = maf_cohort.drop_duplicates('Tumor_Sample_Barcode')[['Tumor_Sample_Barcode', 'purity']]

    # Merging these files
    sample_info = pd.merge(cohort_filt, purity, left_on='tumor_sample', right_on='Tumor_Sample_Barcode')
    sample_info =  pd.merge(sample_info, clinical_filt, left_on='tumor_sample', right_on='Sample ID').drop(['Tumor_Sample_Barcode', 'Sample ID'], axis=1)


    sample_info = sample_info[['sample_id', 'tumor_sample', 'patient', 'Cancer Type', 'Cancer Type Detailed',
                               'Patient Current Age','Sample Type', 'purity','ploidy','Overall Survival (Months)', 
                               'Overall Survival Status','MSI Score', 'MSI Type','Impact TMB Score']]

    sample_info.columns = ['Sample_Id', 'Tumor_Id', 'Patient_Id','Cancer_Type', 'Cancer_Type_Detailed', 'Patient_Current_Age',
                          'Sample_Type', 'purity', 'ploidy', 'Overall_Survival_Months', 'Overall_Survival_Status',
                          'MSI_Score', 'MSI_Type', 'TMB_Score']
    
    return sample_info


In [102]:
create_sample_info(cohort=cohort_type)

,Sample_Id,Tumor_Id,Patient_Id,Cancer_Type,Cancer_Type_Detailed,Patient_Current_Age,Sample_Type,purity,ploidy,Overall_Survival_Months,Overall_Survival_Status,MSI_Score,MSI_Type,TMB_Score
0,P-0025956-T01-IM6_P-0025956-N01-IM6,P-0025956-T01-IM6,P-0025956,Non-Small Cell Lung Cancer,Lung Adenocarcinoma,71.0,Primary,0.273767,3.496971,3.584,DECEASED,0.00,Stable,5.3
1,P-0036909-T01-IM6_P-0036909-N01-IM6,P-0036909-T01-IM6,P-0036909,Non-Small Cell Lung Cancer,Lung Adenocarcinoma,47.0,Metastasis,0.391316,2.871793,14.137,LIVING,0.37,Stable,3.5
2,P-0023546-T01-IM6_P-0023546-N01-IM6,P-0023546-T01-IM6,P-0023546,Prostate Cancer,Prostate Neuroendocrine Carcinoma,50.0,Primary,0.865628,3.115253,4.800,DECEASED,2.37,Stable,3.5
3,P-0023546-T02-IM6_P-0023546-N01-IM6,P-0023546-T02-IM6,P-0023546,Prostate Cancer,Prostate Adenocarcinoma,50.0,Primary,0.312907,3.136841,4.800,DECEASED,0.82,Stable,2.6
4,P-0018837-T01-IM6_P-0018837-N01-IM6,P-0018837-T01-IM6,P-0018837,Colorectal Cancer,Colon Adenocarcinoma,60.0,Primary,0.351778,3.727190,34.060,LIVING,0.45,Stable,5.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9893,P-0050750-T01-IM6_P-0050750-N01-IM6,P-0050750-T01-IM6,P-0050750,Pancreatic Cancer,Pancreatic Adenocarcinoma,55.0,Metastasis,0.301523,3.845451,0.099,LIVING,0.05,Stable,5.3
9894,P-0050643-T01-IM6_P-0050643-N01-IM6,P-0050643-T01-IM6,P-0050643,Breast Cancer,Breast Invasive Ductal Carcinoma,53.0,Primary,0.667936,4.419218,1.315,LIVING,0.30,Stable,4.4
9895,P-0050223-T02-IM6_P-0050223-N02-IM6,P-0050223-T02-IM6,P-0050223,Hepatobiliary Cancer,Cholangiocarcinoma,67.0,Primary,0.215128,4.725790,1.940,LIVING,0.00,Stable,3.5
9896,P-0048760-T01-IM6_P-0048760-N01-IM6,P-0048760-T01-IM6,P-0048760,Bone Cancer,Osteosarcoma,18.0,Primary,0.614603,3.947721,3.682,LIVING,5.79,Indeterminate,6.1


# TP53 Mutations
In this part, we focus on tp53 mutation information.

We gather all mutations per sample, and split it into different columns. We have the following columns:
* Tumor_Id	
* key_1 (2,3,4,5) --> Mutation key allowing to filter duplicates
* vc_1 (2,3,4,5) --> Variant Classification
* ccf_1 (2,3,4,5) --> Cancer Cell Fraction of the mutation
* vaf_1 (2,3,4,5) --> Variant Allele Frequency of the mutation
* HGVSp_1 (2,3,4,5) --> protein change
* spot_1 (2,3,4,5) --> Integer that defines the spot of the tp53 mutation
* tp53_count --> Number of tp53 mutations of the sample

In [104]:
def f_(x):
    # This function helps us to group mutations together in a single cell per patient
    return pd.DataFrame(dict(Tumor_Sample_Barcode = x['Tumor_Sample_Barcode'],  
                        muts = "%s" % ','.join(x['sample_mut_key_vc_ccf_vaf_hgv_spot'])))

def count_tp53_muts(x):
    count = 0
    for i in range(1,6):
        if x['tp53_key_' + str(i)]:
            count+= 1
    return count

cohort = cohort_type


# WARNING: THis function needs sample_info to work
def create_tp53_muts(cohort:str):
    if cohort == 'wgd':
        cohort = cohort_wgd
        maf_cohort = maf_cohort_wgd
        clinical = clinical_wgd
    elif cohort == 'no_wgd':
        cohort = cohort_nowgd
        maf_cohort = maf_cohort_nowgd
        clinical = clinical_nowgd

    '''
    This function aims to gather all tp53 mutation characteristics.
    For each sample we gather the tp53 mutations and their characteristics for all patients.
    '''
    # We load the  table created in maf_tp53_creation.ipynb
    maf_tp53 = maf_cohort[maf_cohort['Hugo_Symbol'] == 'TP53']
    maf_tp53['mut_spot'] = maf_tp53.HGVSp.str.extract('(\d+)')

    # We select only intresting columns
    maf_tp53_filtered = maf_tp53[['Tumor_Sample_Barcode','sample_mut_key', 'Variant_Classification',\
                                        'ccf_expected_copies', 't_var_freq', 'HGVSp','mut_spot' ]]

    # Let's Merge mut_key,Variant_classification, CF, CCF, and VAF to gather them
    maf_tp53_filtered['sample_mut_key_vc_ccf_vaf_hgv_spot'] = maf_tp53_filtered.apply(lambda x: str(x.sample_mut_key)+'%'+str(x.Variant_Classification)+'%'+str(x.ccf_expected_copies)+'%'+str(x.t_var_freq)+'%'+str(x.HGVSp)+'%'+str(x.mut_spot), axis=1)

    # We Select important columns
    final = maf_tp53_filtered[['Tumor_Sample_Barcode', 'sample_mut_key_vc_ccf_vaf_hgv_spot']]
    # We groupby Patient_Id and apply the function above to group mutations
    final = final.groupby(['Tumor_Sample_Barcode'], sort=False).apply(f_)

    # We separate the different mutations into 5 different columns (5 is the max number of tp53 mutations in our cohort)
    final[['mut_key_1','mut_key_2','mut_key_3','mut_key_4','mut_key_5']] = final.muts.str.split(',', expand=True)
    #final = final.drop(['mut_key_6'],axis=1)
    # Split the columns into mut_key_ and vc_
    final[['tp53_key_1','tp53_vc_1','tp53_ccf_1','tp53_vaf_1','tp53_HGVSp_1', 'tp53_spot_1']] = final.mut_key_1.str.split('%', expand=True)
    final[['tp53_key_2','tp53_vc_2','tp53_ccf_2','tp53_vaf_2','tp53_HGVSp_2', 'tp53_spot_2']] = final.mut_key_2.str.split('%', expand=True)
    final[['tp53_key_3','tp53_vc_3','tp53_ccf_3','tp53_vaf_3','tp53_HGVSp_3', 'tp53_spot_3']] = final.mut_key_3.str.split('%', expand=True)
    final[['tp53_key_4','tp53_vc_4','tp53_ccf_4','tp53_vaf_4','tp53_HGVSp_4', 'tp53_spot_4']] = final.mut_key_4.str.split('%', expand=True)
    final[['tp53_key_5','tp53_vc_5','tp53_ccf_5','tp53_vaf_5','tp53_HGVSp_5', 'tp53_spot_5']] = final.mut_key_5.str.split('%', expand=True)

    # We remove the muts column
    final = final.drop(['muts','mut_key_1','mut_key_2','mut_key_3','mut_key_4','mut_key_5'], axis=1)

    # We remove duplicates
    final = final.drop_duplicates('Tumor_Sample_Barcode')

    # We add the cohort patients that are not tp53 positive
    #First we create a dataframe with all missing samples

    cohort_samples = set(sample_info.Tumor_Id)
    final_samples = set(final.Tumor_Sample_Barcode)
    missing_samp = pd.DataFrame(cohort_samples - final_samples, columns = ['Tumor_Sample_Barcode'])
    #Then we append the two datframe
    final = final.append(missing_samp)

    # We rename the Tumor_Sample_Barcode column to have the same key as in other datframes
    final = final.rename(columns={'Tumor_Sample_Barcode': 'Tumor_Id'})

    # We add a last column tp53_count that represents the number of tp53 mutations per sample
    final = final.where(final.notnull(), None)
    final['tp53_count'] = final.apply(count_tp53_muts, axis = 1)

    # We change the type of vafs column to float64 instead of strings
    final = final.astype({'tp53_vaf_1': 'float64', 'tp53_vaf_2': 'float64', 'tp53_vaf_3': 'float64', 'tp53_vaf_4': 'float64', 'tp53_vaf_5': 'float64',
                       'tp53_ccf_1': 'float64', 'tp53_ccf_2': 'float64', 'tp53_ccf_3': 'float64', 'tp53_ccf_4': 'float64', 'tp53_ccf_5': 'float64'})

    return final

In [105]:
create_tp53_muts(cohort=cohort_type)

,Tumor_Id,tp53_key_1,tp53_vc_1,tp53_ccf_1,tp53_vaf_1,tp53_HGVSp_1,tp53_spot_1,tp53_key_2,tp53_vc_2,tp53_ccf_2,tp53_vaf_2,tp53_HGVSp_2,tp53_spot_2,tp53_key_3,tp53_vc_3,tp53_ccf_3,tp53_vaf_3,tp53_HGVSp_3,tp53_spot_3,tp53_key_4,tp53_vc_4,tp53_ccf_4,tp53_vaf_4,tp53_HGVSp_4,tp53_spot_4,tp53_key_5,tp53_vc_5,tp53_ccf_5,tp53_vaf_5,tp53_HGVSp_5,tp53_spot_5,tp53_count
8,P-0036909-T01-IM6,P-0036909-T01-IM617_7577121_G_A,Missense_Mutation,0.798,0.312169,p.Arg273Cys,273,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,1
13,P-0023546-T01-IM6,P-0023546-T01-IM617_7578442_T_C,Missense_Mutation,0.933,0.845070,p.Tyr163Cys,163,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,1
16,P-0023546-T02-IM6,P-0023546-T02-IM617_7578442_T_C,Missense_Mutation,1.000,0.636735,p.Tyr163Cys,163,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,1
21,P-0018837-T01-IM6,P-0018837-T01-IM617_7578406_C_T,Missense_Mutation,1.000,0.325843,p.Arg175His,175,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,1
48,P-0019444-T01-IM6,P-0019444-T01-IM617_7578461_C_A,Missense_Mutation,1.000,0.169002,p.Val157Phe,157,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3190,P-0029543-T01-IM6,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,0
3191,P-0023447-T01-IM6,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,0
3192,P-0002901-T01-IM3,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,0
3193,P-0027887-T01-IM6,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,0


# TP53 Copy Numbers

In this part, we gather the information from gene_level table.
We creaste the following columns:
* Sample_Id 
* tcn --> total copy number
* mcn --> major copy number
* lcn --> lower copy number
* seg_length --> length of the segment
* cn_state --> copy number state
* cf --> Cell fraction of the cn_state
* wgd --> Wholde Genome Doubling (True or False)

In [119]:
# WARNING: THis function needs sample_info to work
def create_copy_number_state(cohort:str):
    if cohort == 'wgd':
        cohort = cohort_wgd
        maf_cohort = maf_cohort_wgd
        clinical = clinical_wgd
        arm_level = arm_level_wgd
        gene_level = gene_level_wgd
    elif cohort == 'no_wgd':
        cohort = cohort_nowgd
        maf_cohort = maf_cohort_nowgd
        clinical = clinical_nowgd
        arm_level = arm_level_nowgd
        gene_level = gene_level_nowgd
    
    # We want TP53 locus so we have to filter the gene
    gene_level = gene_level[gene_level['gene'] == 'TP53']

    gene_level['Tumor_Id'] = gene_level['sample'].str[:17]
    gene_level_subset = gene_level[['sample','tcn','mcn','lcn','seg_length','cn_state', 'cf.em']]
    
    # We rename the cf.em column 
    gene_level_subset = gene_level_subset.rename(columns={'cf.em': 'tp53_cf', 
                                                          'sample':'Sample_Id',
                                                          'tcn': 'tp53_tcn',
                                                          'mcn': 'tp53_mcn',
                                                          'lcn': 'tp53_lcn',
                                                          'seg_length': 'tp53_seg_length',
                                                          'cn_state':'tp53_cn_state'})
    
    # We add WGD information
    wgd = cohort[['sample_id', 'wgd']]
    
    final = pd.merge(gene_level_subset, wgd, left_on='Sample_Id', right_on='sample_id').drop(['sample_id'], axis=1)
    
    
    return final

In [120]:
%%time 
copy_number_info = create_copy_number_state(cohort = cohort_type)
copy_number_info

CPU times: user 19.6 ms, sys: 2.26 ms, total: 21.8 ms
Wall time: 20.4 ms


,Sample_Id,tp53_tcn,tp53_mcn,tp53_lcn,tp53_seg_length,tp53_cn_state,tp53_cf,wgd
0,P-0025956-T01-IM6_P-0025956-N01-IM6,3,2.0,1.0,80668431,LOSS AFTER,0.169170,True
1,P-0036909-T01-IM6_P-0036909-N01-IM6,2,2.0,0.0,18101929,LOSS BEFORE,0.306121,True
2,P-0023546-T01-IM6_P-0023546-N01-IM6,3,3.0,0.0,25250470,CNLOH BEFORE & LOSS,0.835504,True
3,P-0023546-T02-IM6_P-0023546-N01-IM6,2,2.0,0.0,25237770,LOSS BEFORE,0.312907,True
4,P-0018837-T01-IM6_P-0018837-N01-IM6,3,3.0,0.0,25231975,CNLOH BEFORE & LOSS,0.257400,True
...,...,...,...,...,...,...,...,...
9893,P-0050750-T01-IM6_P-0050750-N01-IM6,2,2.0,0.0,13849975,LOSS BEFORE,0.286332,True
9894,P-0050643-T01-IM6_P-0050643-N01-IM6,3,3.0,0.0,11917531,CNLOH BEFORE & LOSS,0.545007,True
9895,P-0050223-T02-IM6_P-0050223-N02-IM6,4,2.0,2.0,80668650,TETRAPLOID,NaN,True
9896,P-0048760-T01-IM6_P-0048760-N01-IM6,3,3.0,0.0,7438283,CNLOH BEFORE & LOSS,0.598599,True


# Computed metrics
In this part we define functions to be applied on the master file to compute specific metrics.


## Genes, mutations and max_vaf
* *create_gene_count*: count of mutated genes for the given sample
* *create_mut_count*: count of mutations for the given sample
* *get_max_vaf*: the maximum of Variant Allele Frequency within all mutation of the sample

In [ ]:
# These 2 first functions allow  to count the total number of genes/mutations per sample.
def create_gene_count(x):
    '''
    This function create the count of genes mutated for each sample.
    Arguments:
        - maf_cohort: the maf_cohort file located in data/merged/data
    '''
    tumor = x.Tumor_Id
    
    selected_cohort = maf_cohort[maf_cohort['Tumor_Sample_Barcode'] == tumor]

    nb_genes = selected_cohort.groupby('Entrez_Gene_Id').size().shape[0]

    return nb_genes

def create_mut_count(x):
    '''
    This function computes the dataframe of all mutation count per sample.
    '''
    tumor = x.Tumor_Id
    selected_cohort = maf_cohort[maf_cohort['Tumor_Sample_Barcode'] == tumor]
    
    return selected_cohort.shape[0]

# The following calculates the max_vaf for a given sample
def get_max_vaf(x):
    tumor = x.Tumor_Id
    selected_cohort = maf_cohort[maf_cohort['Tumor_Sample_Barcode'] == tumor]
    
    return selected_cohort['t_var_freq'].max()

## Genome Instability (frac_genome_aletered, chr_affected by loss, cnkloh, gain )
This script is very long to run, so I decide to run it apart and save the results in a pickle fil that we will merge with our master.

In [ ]:
def compute_frac_genome(x):
    Sample_Id = x.Sample_Id
    wgd_state = x.wgd
    lookup_table = arm_level[arm_level['sample'] == Sample_Id]
    lookup_table['chr'] = lookup_table.arm.str.extract('(\d+)')
    
    if wgd:
        lookup_table_altered = lookup_table[lookup_table['cn_state'] != 'TETRAPLOID'][lookup_table['chr'] != '17']
    else:
        lookup_table_altered = lookup_table[lookup_table['cn_state'] != 'DIPLOID'][lookup_table['chr'] != '17']
        
    altered_length = lookup_table_altered.cn_length.sum()
    total_length = lookup_table.arm_length.sum()
    
    frac_gen_altered = round(altered_length/total_length,3)
    
    return frac_gen_altered

# Here is the function that allws to compute genome instability columns
def chr_computations(x):
    CNLOH = ['CNLOH', 'CNLOH BEFORE & LOSS', 'CNLOH AFTER', 'CNLOH BEFORE', 'CNLOH & GAIN', 'CNLOH BEFORE & GAIN', 'AMP (LOH)']
    LOSS = ['HETLOSS', 'LOSS BEFORE', 'LOSS AFTER', 'HOMDEL', 'LOSS BEFORE & AFTER', 'DOUBLE LOSS AFTER', 'LOSS & GAIN', 'CNLOH BEFORE & LOSS']
    GAIN = ['GAIN', 'AMP', 'AMP (BALANCED)', 'LOSS & GAIN', 'CNLOH & GAIN', 'CNLOH BEFORE & GAIN']
    
    arm_level_samples = list(set(arm_level['sample']))
    
    if x.Sample_Id not in arm_level_samples:
        return ['NaN','NaN','NaN','NaN', 'NaN']
    
    lookup_table = arm_level[arm_level['sample'] == x.Sample_Id]
    lookup_table['chr'] = lookup_table.arm.str.extract('(\d+)')
    
    if wgd:
        lookup_table = lookup_table[lookup_table['cn_state'] != 'TETRAPLOID'][lookup_table['chr'] != '17']
    else:
        lookup_table = lookup_table[lookup_table['cn_state'] != 'DIPLOID'][lookup_table['chr'] != '17']
    lookup_table['state_chr'] = lookup_table['cn_state']+lookup_table['chr']
    
    # If only DIPLOID or TETRAPLOID
    if lookup_table.empty:
        return [float(0)]*5
    
    lookup_table['cnloh_chr'] = lookup_table.apply(condition_CNLOH, axis=1)
    lookup_table['loss_chr'] = lookup_table.apply(condition_LOSS, axis=1)
    lookup_table['gain_chr'] = lookup_table.apply(condition_GAIN, axis=1)

    #chr_affected colum
    lookup_table_chr = lookup_table.drop_duplicates(subset=['chr'])
    chr_affected = len(lookup_table_chr)
    
    #chr_loss, chr_gain, chr_cnloh columns
    lookup_table_cnloh = lookup_table.drop_duplicates(subset=['cnloh_chr'])['cnloh_chr']
    lookup_table_loss = lookup_table.drop_duplicates(subset=['loss_chr'])['loss_chr']
    lookup_table_gain = lookup_table.drop_duplicates(subset=['gain_chr'])['gain_chr']

    chr_loss = len(lookup_table_loss[lookup_table_loss.str.startswith('LOSS')])
    chr_gain = len(lookup_table_gain[lookup_table_gain.str.startswith('GAIN')])
    chr_cnloh = len(lookup_table_cnloh[lookup_table_cnloh.str.startswith('CNLOH')])
    
    #frac_gen_altered column
    frac_gen_altered = compute_frac_genome(x, arm_level)
    
    return [chr_affected, chr_loss, chr_gain, chr_cnloh, frac_gen_altered]

## TP53 Residual

In [ ]:
# The following function needs to be called on the complete master file because it needs info from different parts
# It computes the expected number of tp53 mutant copies in a cell
def create_copies_tp53_muts(master):
    master['tp53_exp_nb_1'] = master.apply(lambda x:(x.tp53_vaf_1 / x.purity) * (x.tp53_tcn * x.purity + 2*(1 - x.purity)), axis = 1)
    master['tp53_exp_nb_2'] = master.apply(lambda x:(x.tp53_vaf_2 / x.purity) * (x.tp53_tcn * x.purity + 2*(1 - x.purity)), axis = 1)
    master['tp53_exp_nb_3'] = master.apply(lambda x:(x.tp53_vaf_3 / x.purity) * (x.tp53_tcn * x.purity + 2*(1 - x.purity)), axis = 1)
    master['tp53_exp_nb_4'] = master.apply(lambda x:(x.tp53_vaf_4 / x.purity) * (x.tp53_tcn * x.purity + 2*(1 - x.purity)), axis = 1)
    master['tp53_exp_nb_5'] = master.apply(lambda x:(x.tp53_vaf_5 / x.purity) * (x.tp53_tcn * x.purity + 2*(1 - x.purity)), axis = 1)
    
    return master


# The following computes the expected number of copies of tp53 residuals 
def create_tp53_res(master):
    master['tp53_res_1'] = master.apply(lambda x:x.tcn - x.tp53_exp_nb_1, axis = 1)
    master['tp53_res_2'] = master.apply(lambda x:x.tcn - x.tp53_exp_nb_2, axis = 1)
    master['tp53_res_3'] = master.apply(lambda x:x.tcn - x.tp53_exp_nb_3, axis = 1)
    master['tp53_res_4'] = master.apply(lambda x:x.tcn - x.tp53_exp_nb_4, axis = 1)
    master['tp53_res_5'] = master.apply(lambda x:x.tcn - x.tp53_exp_nb_5, axis = 1)
    
    return master

## Mutation Types Grouping

In [ ]:
# The following functions allow to group the Mutation Types
def vc_group_cond_1(x):
    truncated = ['Splice_Site','Intron','Nonsense_Mutation','Splice_Region','Frame_Shift_Del','Frame_Shift_Ins']
    in_frame = ['In_Frame_Ins','In_Frame_Del']
    missense = ['Missense_Mutation']
    
    if x.tp53_vc_1 in truncated: return 'truncated'
    if x.tp53_vc_1 in in_frame: return 'in_frame'
    if x.tp53_vc_1 in missense: 
        if x.tp53_spot_1 in ['273','248','175']: return x.tp53_spot_1
        elif x.tp53_spot_1 in ['245', '282', '213', '352', '220', '196']: return 'hotspot'
        else: return 'missense'
def vc_group_cond_2(x):
    truncated = ['Splice_Site','Intron','Nonsense_Mutation','Splice_Region','Frame_Shift_Del','Frame_Shift_Ins']
    in_frame = ['In_Frame_Ins','In_Frame_Del']
    missense = ['Missense_Mutation']
    
    if x.tp53_vc_2 in truncated: return 'truncated'
    if x.tp53_vc_2 in in_frame: return 'in_frame'
    if x.tp53_vc_2 in missense: 
        if x.tp53_spot_2 in ['273','248','175']: return x.tp53_spot_2
        elif x.tp53_spot_2 in['245', '282', '213', '352', '220', '196']: return 'hotspot'
        else: return 'missense'   
def vc_group_cond_3(x):
    truncated = ['Splice_Site','Intron','Nonsense_Mutation','Splice_Region','Frame_Shift_Del','Frame_Shift_Ins']
    in_frame = ['In_Frame_Ins','In_Frame_Del']
    missense = ['Missense_Mutation']
    
    if x.tp53_vc_3 in truncated: return 'truncated'
    if x.tp53_vc_3 in in_frame: return 'in_frame'
    if x.tp53_vc_3 in missense: 
        if x.tp53_spot_3 in ['273','248','175']: return x.tp53_spot_3
        elif x.tp53_spot_3 in ['245', '282', '213', '352', '220', '196']: return 'hotspot'
        else: return 'missense' 
def vc_group_cond_4(x):
    truncated = ['Splice_Site','Intron','Nonsense_Mutation','Splice_Region','Frame_Shift_Del','Frame_Shift_Ins']
    in_frame = ['In_Frame_Ins','In_Frame_Del']
    missense = ['Missense_Mutation']
    
    if x.tp53_vc_4 in truncated: return 'truncated'
    if x.tp53_vc_4 in in_frame: return 'in_frame'
    if x.tp53_vc_4 in missense: 
        if x.tp53_spot_4 in ['273','248','175']: return x.tp53_spot_4
        elif x.tp53_spot_4 in ['245', '282', '213', '352', '220', '196']: return 'hotspot'
        else: return 'missense'
def vc_group_cond_5(x):
    truncated = ['Splice_Site','Intron','Nonsense_Mutation','Splice_Region','Frame_Shift_Del','Frame_Shift_Ins']
    in_frame = ['In_Frame_Ins','In_Frame_Del']
    missense = ['Missense_Mutation']
    
    if x.tp53_vc_5 in truncated: return 'truncated'
    if x.tp53_vc_5 in in_frame: return 'in_frame'
    if x.tp53_vc_5 in missense: 
        if x.tp53_spot_5 in ['273','248','175']: return x.tp53_spot_5
        elif x.tp53_spot_5 in ['245', '282', '213', '352', '220', '196']: return 'hotspot'
        else: return 'missense'

## Non-WGD specific functions

* *cn_group_cond*

First, we group the different Copy Number States *cn_state* in subgroups, under the column *cn_group*:
    * Group 1: cnLOH gathering ['CNLOH']
    * Group 2: LOSS gathering ['HETLOSS']
    * Group 3: HOMDEL gathering ['HOMDEL']
    * Group 4: WILD_TYPE gathering ['DIPLOID', 'TETRAPLOID']
    * Group 5: GAIN gathering ['GAIN']
    * Group 6: OTHER gathering ['AMP (BALANCED)', 'AMP (LOH)', 'AMP','LOSS & GAIN', 'CNLOH & GAIN']

* *mut_cn_group_cond*

Based on this first column we define 7 final groups of patients adding the mutational information. These groups will be under the column *tp53_group*.
    * Group 1: Samples with 0 tp53 mutations and HETLOSS
    * Group 2: Samples with HOMDEL
    * Group 3: Samples with 1 tp53 mutation and WILD_TYPE (DIPLOID, LOSS AFTER, TETRAPLOID)
    * Group 4: Samples with 1 tp53 mutation or more and LOSS
    * Group 5: Samples with 1 tp53 mutation or more and cnLOH
    * Group 6: Samples with 2/3/4/5 tp53 mutations and WILD_TYPE or GAIN

* *tp53_residual_group*: defines if we have residual tp53 or not based on the expected WT tp53 residual


* *get_loh_nowgd*

The last function is meant to compute the LOH state

In [147]:
def cn_group_cond(x):
    if x.tp53_cn_state in ['CNLOH']:
        return 'cnLOH'
    if x.tp53_cn_state in ['HETLOSS']:
        return 'LOSS'
    if x.tp53_cn_state == 'HOMDEL':
        return 'HOMDEL'
    if x.tp53_cn_state in ['DIPLOID', 'TETRAPLOID']:
        return 'WILD_TYPE'
    if x.tp53_cn_state == 'GAIN':
        return 'GAIN'
    if x.tp53_cn_state in ['AMP (BALANCED)', 'AMP (LOH)', 'AMP','LOSS & GAIN', 'CNLOH & GAIN']:
        return 'OTHER'

def mut_cn_group_cond(x):
    if x.tp53_cn_state == 'HETLOSS' and x.tp53_count == 0:
        return '0_HETLOSS'
    if x.tp53_first_group == 'HOMDEL':
        return 'HOMDEL'
    if x.tp53_first_group == 'WILD_TYPE' and x.tp53_count == 1 :
        return '1_WILD_TYPE'
    if x.tp53_first_group == 'LOSS' and x.tp53_count >=1:
        return '>=1_LOSS'
    if x.tp53_first_group == 'cnLOH' and x.tp53_count >=1:
        return '>=1_cnLOH'
    if (x.tp53_first_group == 'WILD_TYPE' or x.tp53_first_group == 'GAIN') and x.tp53_count > 1:
        return '>1muts'
    
    
def tp53_residual_group(x):
    if x.tp53_group == '1_WILD_TYPE' or x.tp53_group == '0_HETLOSS':
        return 'tp53_res'
    if x.tp53_group == 'HOMDEL':
        return 'no_tp53_res'
    if x.tp53_group == '>=1_LOSS' or x.tp53_group == '>=1_cnLOH':
        if (x.tp53_residual_1 < 0.5) or (x.tp53_residual_2 < 0.5):
            return 'no_tp53_res'
        else:
            if (x.tp53_cf + max(x.tp53_ccf_1, x.tp53_ccf_2, x.tp53_ccf_3, x.tp53_ccf_4, x.tp53_ccf_5)) > 1:
                return 'no_tp53_res'
            else:
                return 'uncertain'
    if x.tp53_group == '>1muts':
        if (x.tp53_residual_1 + x.tp53_residual_2 < 2.5):
            if (x.tp53_ccf_1 + x.tp53_ccf_2 > 1):
                return 'no_tp53_res'
            else: 
                return 'uncertain'
           
        elif (x.tp53_residual_1 + x.tp53_residual_2 > 2.5):
            return 'tp53_res'
        
        
def get_loh_nowgd(x):
    if tp53_cn_state in ['CNLOH', 'CNLOH & GAIN', 'AMP (LOH)']:
        return True
    else: return False

## WGD specific functions
Finally we have functions to compute TP53 allelic state BEFORE WGD (for the WGD cohort only):
* *get_bi_nobi* allows to say if the pre WGD allelic state was Bi-Allelic or not
* *get_mono* allows to say if, within Not Bi-allelic samples, the state was mono-allelic or with 2 WT alleles
* *get_loh_wgd* that computes if the samples is in LOH state or not

In [ ]:
'''
            NUMBER OF TP53 residual ASSOCIATED WITH ALLELIC STATE BEFORE WGD
            
            
                                     BI-ALLELIC | MONO - ALLELIC | 2WT

                LOSS BEFORE :            0      |        1       | -
                CNLOH BEFORE & LOSS:     0      |       1,2      | 2
                CNLOH BEFORE:            0      |        2       | 3
                LOSS AFTER:              -      |       1,2      | 2
                DOUBLE LOSS AFTER:       -      |       0,1      | 1
                TETRAPLOID:              -      |        2       | 3
                CNLOH AFTER:             -      |       1,3      | 3
                CNLOH BEFORE & GAIN:     0      |        2,3     | 3,4
                

                        THRESHOLDS BETWEEN ALLELIC STATES BEFORE WGD
                    
                                     BI-ALLELIC | MONO - ALLELIC | 2WT

                LOSS BEFORE :            <0.4   |        >0.6    | -
                CNLOH BEFORE & LOSS:     <0.4   |      0.6< <1.5 | uncertain
                CNLOH BEFORE:            <1.5   |   1.5< <2.5    | >2.5
                LOSS AFTER:              -      |      <1.5      | uncertain 
                DOUBLE LOSS AFTER:       -      |       <0.5     | uncertain
                TETRAPLOID:              -      |       <2.5     | >2.5
                CNLOH AFTER:             -      |       <1.5     | uncertain
                CNLOH BEFORE & GAIN:     <1.4   |   1.6< <2.5    | uncertain
                
    

 '''


# The following functions are for WGD cohort, computing the tp53 allelic state before WGD
def get_bi_nobi(x):
    tumor = x.tumor_sample
    cn_state = x.tp53_cn_state
    tp53_count = x.tp53_count
    maf_muts = maf_cohort[maf_cohort['Tumor_Sample_Barcode'] == tumor]
    nb_tp53muts = maf_muts.shape[0]
    
    # Samples with only one tp53 mutation
    if nb_tp53muts == 1:
        tp53_res = maf_muts.tp53_res.values[0]
        
        if cn_state == 'LOSS BEFORE' or cn_state == 'CNLOH BEFORE & LOSS':
            thr = 0.5
            if tp53_res < thr - 0.1:
                return 'bi'
            elif (tp53_res < thr + 0.1) and (tp53_res > thr - 0.1):
                return 'uncertain'
            elif tp53_res > thr + 0.1:
                return 'no_bi'

        elif cn_state == 'CNLOH BEFORE':
            thr = 1.5
            if tp53_res < thr:
                return 'bi'
            elif tp53_res >= thr:
                return 'no_bi'

        elif cn_state in ['LOSS AFTER','DOUBLE LOSS AFTER','TETRAPLOID','CNLOH AFTER']: 
            return 'no_bi'

        elif cn_state == 'CNLOH BEFORE & GAIN':
            thr = 1.5
            if tp53_res < thr - 0.1:
                return 'bi'
            elif (tp53_res < thr + 0.1) and (tp53_res > thr - 0.1):
                return 'uncertain'
            elif tp53_res > thr + 0.1:
                return 'no_bi'
            
            
            
    # Samples with 2 tp53 mutations
    elif nb_tp53muts == 2:
        tp53_res_1 = maf_muts.tp53_res.values[0]
        tp53_res_2 = maf_muts.tp53_res.values[1]
        timing_1 = maf_muts.tp53_timing_wgd.values[0]
        timing_2 = maf_muts.tp53_timing_wgd.values[1]
        
        if cn_state == 'LOSS BEFORE':
            thr = 0.5
            if (tp53_res_1 < thr - 0.1) or (tp53_res_2 < thr - 0.1):
                return 'bi'
            elif (tp53_res_1 > thr + 0.1) and (tp53_res_2 > thr + 0.1):
                return 'no_bi'
            else: return 'uncertain'
            
        if cn_state == 'CNLOH BEFORE & LOSS':
            thr_1 = 0.5
            thr_2 = 1.5
            if (tp53_res_1 < thr_1 - 0.1) or (tp53_res_2 < thr_1 - 0.1):
                return 'bi'
            elif (tp53_res_1 < thr_2 and tp53_res_2 < thr_2):
                return 'bi'
            else: return 'no_bi'
            
        elif cn_state == 'CNLOH BEFORE':
            thr_1 = 1.5
            thr_2 = 2.5
            if (tp53_res_1 < thr_1) or (tp53_res_2 < thr_1):
                return 'bi'
            elif (tp53_res_1 < thr_2 and tp53_res_2 < thr_2):
                return 'bi'
            else: return 'no_bi'

        elif cn_state =='LOSS AFTER': 
            thr = 1.5
            if (tp53_res_1 < thr) and (tp53_res_2 < thr):
                return 'bi'
            else: return 'no_bi'
            
        elif cn_state =='DOUBLE LOSS AFTER': 
            thr = 0.5
            if (tp53_res_1 < thr) and (tp53_res_2 < thr):
                return 'bi'
            else: return 'no_bi'
        
        elif cn_state =='TETRAPLOID': 
            thr = 2.5
            if (tp53_res_1 < thr) and (tp53_res_2 < thr):
                return 'bi'
            else: return 'no_bi'
            
        elif cn_state =='CNLOH AFTER': 
            thr = 1.5
            if (tp53_res_1 < thr) and (tp53_res_2 < thr):
                return 'bi'
            else: return 'no_bi'
        
        elif cn_state == 'CNLOH BEFORE & GAIN':
            thr_1 = 1.5
            thr_2 = 2.5
            if tp53_res_1 < thr_1 - 0.1 or tp53_res_2 < thr_1 - 0.1:
                return 'bi'
            elif (tp53_res_1 > thr_2) and (tp53_res_2 > thr_2):
                return 'no_bi'
            else: return 'uncertain'
            
            
    # Samples with 3 tp53 mutations
    elif nb_tp53muts == 3:
        tp53_res_1 = maf_muts.tp53_res.values[0]
        tp53_res_2 = maf_muts.tp53_res.values[1]
        tp53_res_3 = maf_muts.tp53_res.values[2]
        tp53_res = [tp53_res_1,tp53_res_2,tp53_res_3]
        tp53_res.sort()
        
        timing_1 = maf_muts.tp53_timing_wgd.values[0]
        timing_2 = maf_muts.tp53_timing_wgd.values[1]
        timing_3 = maf_muts.tp53_timing_wgd.values[2]
        timings = [timing_1, timing_2, timing_3]
        BEF = [1 for x in timings if x=='BEFORE']
        
        # Number of mutations happening before wgd
        nb_bef = len(BEF)
        
        if cn_state == 'LOSS BEFORE':
            thr = 0.5
            if (min(tp53_res) < thr - 0.1):
                return 'bi'
            elif (max(tp53_res) > thr + 0.1) :
                return 'no_bi'
            else: return 'uncertain'
            
        if cn_state == 'CNLOH BEFORE & LOSS':
            thr_1 = 0.5
            thr_2 = 1.5
            if (min(tp53_res) < thr_1 - 0.1):
                return 'bi'
            elif (tp53_res[1] < thr_2): # we want that exactly 2 mutation arose before WGD
                return 'bi'
            else: return 'no_bi'
            
        elif cn_state == 'CNLOH BEFORE':
            thr_1 = 1.5
            thr_2 = 2.5
            if (min(tp53_res) < thr_1):
                return 'bi'
            elif (tp53_res[1] < thr_2):
                return 'bi'
            else: return 'no_bi'

        elif cn_state =='LOSS AFTER': 
            thr = 1.5
            if (tp53_res[1] < thr):
                return 'bi'
            else: return 'no_bi'
            
        elif cn_state =='DOUBLE LOSS AFTER': 
            thr = 0.5
            if (tp53_res[1] < thr):
                return 'bi'
            else: return 'no_bi'
        
        elif cn_state =='TETRAPLOID': 
            thr = 2.5
            if (tp53_res[1] < thr):
                return 'bi'
            else: return 'no_bi'
            
        elif cn_state =='CNLOH AFTER': 
            thr = 1.5
            if (tp53_res[1] < thr):
                return 'bi'
            else: return 'no_bi'
        
        elif cn_state == 'CNLOH BEFORE & GAIN':
            thr_1 = 1.5
            thr_2 = 2.5
            if min(tp53_res) < thr_1 - 0.1 :
                return 'bi'
            elif (tp53_res[1] > thr_2):
                return 'no_bi'
            else: return 'uncertain'
    
    else: return 'uncertain'


# This functions can only be called after get_bi_nobi() has been called on master
def get_mono(x):
    tumor = x.tumor_sample
    cn_state = x.tp53_cn_state
    tp53_count = x.tp53_count
    maf_muts = maf_cohort[maf_cohort['Tumor_Sample_Barcode'] == tumor]
    nb_tp53muts = maf_muts.shape[0]
    bi_state = x.bi_state_wgd
    
    if bi_state == 'bi':
        return 'bi'
    
    elif bi_state == 'uncertain':
        return 'uncertain_bi'
    
    
    elif bi_state == 'no_bi': # We are already in the  mono/2WT distinction 
        
        #1 mut samples
        if nb_tp53muts == 1:
            tp53_res = maf_muts.tp53_res.values[0]
            
            if cn_state == 'LOSS BEFORE':
                return 'mono'
            elif cn_state == 'CNLOH BEFORE' or cn_state == 'TETRAPLOID':
                if tp53_res < 2.5:
                    return 'mono'
                elif tp53_res > 2.5:
                    return '2WT'
                
            else: return 'uncertain_mono'

        
        # 2 mut samples   
        elif nb_tp53muts == 2:
            tp53_res_1 = maf_muts.tp53_res.values[0]
            tp53_res_2 = maf_muts.tp53_res.values[1]
            
            if cn_state == 'LOSS BEFORE':
                return 'mono'
            elif cn_state == 'CNLOH BEFORE' or cn_state == 'TETRAPLOID':
                if tp53_res_1 < 2.5 or tp53_res_2 < 2.5:
                    return 'mono'
                elif tp53_res_1 > 2.5 and tp53_res_2 > 2.5:
                    return '2WT'
                
            else: return 'uncertain_mono'
            
        # 3 mut samples
        elif nb_tp53muts == 3:
            tp53_res_1 = maf_muts.tp53_res.values[0]
            tp53_res_2 = maf_muts.tp53_res.values[1]
            tp53_res_3 = maf_muts.tp53_res.values[2]
            tp53_res = [tp53_res_1,tp53_res_2,tp53_res_3]
            tp53_res.sort()
            
            if cn_state == 'LOSS BEFORE':
                return 'mono'
            elif cn_state == 'CNLOH BEFORE' or cn_state == 'TETRAPLOID':
                if min(tp53_res) < 2.5:
                    return 'mono'
                elif tp53_res[0] > 2.5:
                    return '2WT'
                
            else: return 'uncertain_mono'
        
        else: return 'uncertain_mono' 
        


# Merge Tables

In [123]:
def merge_tables(cohort_type: str):
    
    if cohort == 'wgd':
        cohort = cohort_wgd
        maf_cohort = maf_cohort_wgd
        clinical = clinical_wgd
        arm_level = arm_level_wgd
        gene_level = gene_level_wgd
    elif cohort == 'no_wgd':
        cohort = cohort_nowgd
        maf_cohort = maf_cohort_nowgd
        clinical = clinical_nowgd
        arm_level = arm_level_nowgd
        gene_level = gene_level_nowgd
    
    sample_info = create_sample_info(cohort=cohort_type)
    tp53_muts = create_tp53_muts(cohort=cohort_type)
    copy_number_info = create_copy_number_state(cohort=cohort_type)
    
    master = pd.merge(sample_info, tp53_muts, on='Tumor_Id')
    master = pd.merge(master, copy_number_info, on='Sample_Id')
    
    return master

In [125]:
%%time
master = merge_tables(cohort_type=cohort_type)

CPU times: user 5.65 s, sys: 71.6 ms, total: 5.73 s
Wall time: 5.7 s


In [ ]:
def get_max_vaf(x):
    tumor = x.Tumor_Id
    selected_cohort = maf_cohort[maf_cohort['Tumor_Sample_Barcode'] == tumor]
    
    return selected_cohort['t_var_freq'].max()

In [140]:
master['gene_count'] = master.apply(create_gene_count, axis=1)

In [141]:
master

,Sample_Id,Tumor_Id,Patient_Id,Cancer_Type,Cancer_Type_Detailed,Patient_Current_Age,Sample_Type,purity,ploidy,Overall_Survival_Months,Overall_Survival_Status,MSI_Score,MSI_Type,TMB_Score,tp53_key_1,tp53_vc_1,tp53_ccf_1,tp53_vaf_1,tp53_HGVSp_1,tp53_spot_1,tp53_key_2,tp53_vc_2,tp53_ccf_2,tp53_vaf_2,tp53_HGVSp_2,tp53_spot_2,tp53_key_3,tp53_vc_3,tp53_ccf_3,tp53_vaf_3,tp53_HGVSp_3,tp53_spot_3,tp53_key_4,tp53_vc_4,tp53_ccf_4,tp53_vaf_4,tp53_HGVSp_4,tp53_spot_4,tp53_key_5,tp53_vc_5,tp53_ccf_5,tp53_vaf_5,tp53_HGVSp_5,tp53_spot_5,tp53_count,tp53_tcn,tp53_mcn,tp53_lcn,tp53_seg_length,tp53_cn_state,tp53_cf,wgd,mut_count,gene_count
0,P-0025956-T01-IM6_P-0025956-N01-IM6,P-0025956-T01-IM6,P-0025956,Non-Small Cell Lung Cancer,Lung Adenocarcinoma,71.0,Primary,0.273767,3.496971,3.584,DECEASED,0.00,Stable,5.3,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,0,3,2.0,1.0,80668431,LOSS AFTER,0.169170,True,6,6
1,P-0036909-T01-IM6_P-0036909-N01-IM6,P-0036909-T01-IM6,P-0036909,Non-Small Cell Lung Cancer,Lung Adenocarcinoma,47.0,Metastasis,0.391316,2.871793,14.137,LIVING,0.37,Stable,3.5,P-0036909-T01-IM617_7577121_G_A,Missense_Mutation,0.798,0.312169,p.Arg273Cys,273,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,1,2,2.0,0.0,18101929,LOSS BEFORE,0.306121,True,4,3
2,P-0023546-T01-IM6_P-0023546-N01-IM6,P-0023546-T01-IM6,P-0023546,Prostate Cancer,Prostate Neuroendocrine Carcinoma,50.0,Primary,0.865628,3.115253,4.800,DECEASED,2.37,Stable,3.5,P-0023546-T01-IM617_7578442_T_C,Missense_Mutation,0.933,0.845070,p.Tyr163Cys,163,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,1,3,3.0,0.0,25250470,CNLOH BEFORE & LOSS,0.835504,True,4,4
3,P-0023546-T02-IM6_P-0023546-N01-IM6,P-0023546-T02-IM6,P-0023546,Prostate Cancer,Prostate Adenocarcinoma,50.0,Primary,0.312907,3.136841,4.800,DECEASED,0.82,Stable,2.6,P-0023546-T02-IM617_7578442_T_C,Missense_Mutation,1.000,0.636735,p.Tyr163Cys,163,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,1,2,2.0,0.0,25237770,LOSS BEFORE,0.312907,True,3,3
4,P-0018837-T01-IM6_P-0018837-N01-IM6,P-0018837-T01-IM6,P-0018837,Colorectal Cancer,Colon Adenocarcinoma,60.0,Primary,0.351778,3.727190,34.060,LIVING,0.45,Stable,5.3,P-0018837-T01-IM617_7578406_C_T,Missense_Mutation,1.000,0.325843,p.Arg175His,175,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,1,3,3.0,0.0,25231975,CNLOH BEFORE & LOSS,0.257400,True,6,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9893,P-0050750-T01-IM6_P-0050750-N01-IM6,P-0050750-T01-IM6,P-0050750,Pancreatic Cancer,Pancreatic Adenocarcinoma,55.0,Metastasis,0.301523,3.845451,0.099,LIVING,0.05,Stable,5.3,P-0050750-T01-IM617_7578406_C_T,Missense_Mutation,0.946,0.285229,p.Arg175His,175,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,1,2,2.0,0.0,13849975,LOSS BEFORE,0.286332,True,5,5
9894,P-0050643-T01-IM6_P-0050643-N01-IM6,P-0050643-T01-IM6,P-0050643,Breast Cancer,Breast Invasive Ductal Carcinoma,53.0,Primary,0.667936,4.419218,1.315,LIVING,0.30,Stable,4.4,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,0,3,3.0,0.0,11917531,CNLOH BEFORE & LOSS,0.545007,True,5,5
9895,P-0050223-T02-IM6_P-0050223-N02-IM6,P-0050223-T02-IM6,P-0050223,Hepatobiliary Cancer,Cholangiocarcinoma,67.0,Primary,0.215128,4.725790,1.940,LIVING,0.00,Stable,3.5,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,None,None,NaN,NaN,None,None,0,

In [145]:
maf_cohort_wgd[maf_cohort_wgd['Tumor_Sample_Barcode'] == 'P-0036909-T01-IM6']['t_var_freq'].max()

0.31216931216931204

In [137]:
selected_cohort = maf_cohort[maf_cohort['Tumor_Sample_Barcode'] == 'P-0025956-T01-IM6']
selected_cohort.groupby('Entrez_Gene_Id').size().shape[0]

6